# Resume Q&A Chatbot 🗒️🤖

The goal of this notebook is to create a command line chat interface such that if you give it the path of your resume, you should be able to ask questions about your resume by chatting with the chatbot.


### Step 1: Import the Required Packages and load the ENV Vars


In [58]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage


load_dotenv(
    "/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/04-Resume-ChatBot/.env"
)

True

### Step 2: Initialize the Embedding Model and the LLM


In [59]:
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-4o-mini")

### Step 3: Use the path of the resume to create a vector database

#### Load the file


In [60]:
resume_file_path = "/Users/aryankhurana/Developer/GenAI-with-Langchain/projects/04-Resume-ChatBot/test.pdf"

loader = None
if resume_file_path.endswith(".pdf"):
    loader = PyPDFLoader(resume_file_path)
else:
    loader = TextLoader(resume_file_path)

documents = loader.load()
print(len(documents))

1


#### Split the documents into chunks


In [63]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=99999999999, chunk_overlap=200, add_start_index=True
)
chunks = text_splitter.split_documents(documents)
len(chunks)

1

#### Create a vector database


In [64]:
client = QdrantClient(":memory:")


vector_db = Qdrant.from_documents(
    documents=chunks,
    embedding=embeddings,
    location=":memory:",
    collection_name="resume_collection",
)

### Step 04: Create a function to create a chain


In [65]:
def create_chain(vector_db: Qdrant):
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    template = """
    You are a helpful assistant that ONLY answers questions about the resume information provided below.
    
    RULES:
    1. Only answer questions that are directly related to the resume information
    2. If the question is not about the resume or the person's professional background, politely decline to answer
    3. Do not make up any information that is not in the resume
    4. Be concise and professional in your responses
    5. If the information requested is not in the context, say you don't have that information in the resume
    
    RESUME CONTEXT:
    {context}
    
    CONVERSATION HISTORY:
    {chat_history}
    
    QUESTION: {question}
    
    YOUR ANSWER:
    """

    prompt = ChatPromptTemplate.from_template(template)

    def get_context(input_dict):
        question = input_dict["question"]
        if isinstance(question, str):
            return retriever.invoke(question)
        else:
            return ["Error: Invalid question format"]

    chain = (
        {
            "context": get_context,
            "chat_history": lambda x: x["chat_history"],
            "question": lambda x: x["question"],
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

### Step 05: Create a function to filter irrelevant queries


In [ ]:
# def filter_irrelevant_query(query: str) -> bool:
#     filter_template = """
#     Determine if the following question is asking about information that could be found in a resume,
#     about someone's professional experience, education, skills, or background.

#     QUESTION: {question}

#     Answer only YES or NO:
#     """

#     filter_prompt = ChatPromptTemplate.from_template(filter_template)
#     filter_chain = filter_prompt | llm | StrOutputParser()

#     result = filter_chain.invoke({"question": query})
#     return "YES" in result.upper()

### Step 06: Create the main chatbot function


In [70]:
def resume_chatbot(query: str, chain, memory) -> str:
    # if not filter_irrelevant_query(query):
    #     response = "I'm designed to only answer questions about the resume. This doesn't appear to be related to the resume information. Is there something specific about the professional background you'd like to know?"
    #     memory.append(HumanMessage(content=query))
    #     memory.append(AIMessage(content=response))
    #     return response

    chat_history = ""
    for i in range(0, len(memory), 2):
        if i + 1 < len(memory):
            chat_history += f"Human: {memory[i].content}\n"
            chat_history += f"AI: {memory[i+1].content}\n\n"

    input_data = {"question": query, "chat_history": chat_history}

    try:
        response = chain.invoke(input_data)
    except Exception as e:
        print(f"Error in chain: {e}")
        response = (
            "I'm having trouble processing your question. Could you try rephrasing it?"
        )

    memory.append(HumanMessage(content=query))
    memory.append(AIMessage(content=response))

    return response

### Step 07 (Final Step): Bring everything together


In [68]:
chain = create_chain(vector_db)

In [72]:
memory = []
print(
    "Resume chatbot initialized. Ask a question about the resume or the person's professional background."
)

while True:
    query = input("You: ")

    if query.lower() == "exit":
        print("Exiting chatbot...")
        break

    response = resume_chatbot(query, chain, memory)
    print(f"\nChatbot: {response}")
    print()

Resume chatbot initialized. Ask a question about the resume or the person's professional background.

Chatbot: Aryan Khurana worked on the following projects:

1. **Fragments**: A universal conversion microservice that transforms any text, application, or media file into multiple formats, utilizing technologies such as Express.js, AWS (S3, DynamoDB, ECS, ECR, Cognito), Docker, GitHub Actions, Jest, and Ansible.

2. **WattWise**: An IoT-powered energy monitoring system that aggregates live power data, predicts monthly costs, and flags anomalies via AI. This project won 2nd place at Hackthe6ix 2024 and utilized technologies including Express.js, MQTT, Docker, DynamoDB, Next.js, Flask, GitHub Actions, scikit-learn, and Google Gemini GenAI.


Chatbot: I'm here to assist with questions related to the resume information provided. Please let me know if you have any specific inquiries regarding Aryan Khurana's education, work experience, projects, technical skills, or achievements.

Exiting ch